<a href="https://colab.research.google.com/github/kimpitang/AI-Agent/blob/main/%EC%97%90%EC%9D%B4%EC%A0%84%ED%8A%B8%EC%97%90_%EA%B8%B0%EC%96%B5%EB%A0%A5_%EB%A7%8C%EB%93%A4%EC%96%B4%EC%A3%BC%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ""
os.environ['TAVILY_API_KEY'] = ""

In [ ]:
!pip install langgraph
!pip install langchain-community
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:

### 기억력 만들기

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver() # 인메모리라서 서버 구축에는 다른 MemorySaver사용 필요

class State(TypedDict):
  messages: Annotated[list, add_messages]

tool = TavilySearchResults(max_results=2)
tools = [tool]
tool_node = ToolNode(tools)

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
  result = llm_with_tools.invoke(state["messages"])
  return {"messages": [result]}

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "1"}}
while True:
  user_input = input("User: ")
  if user_input.lower() in ["quit", "exit", "q"]:
    print("Goodbye!")
    break
  for event in graph.stream({"messages": ("user", user_input)}, config):
    for value in event.values():
      print("Assistant:", value["messages"][-1].content)

User: 대한민국의 대통령은 누구야?
Assistant: 
Assistant: [{"url": "https://namu.wiki/w/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%20%EB%8C%80%ED%86%B5%EB%A0%B9", "content": "대한민국의 현직 대통령은 제20대 윤석열이며, 임기는 2022년 5월 10일부터 2027년 5월 9일까지이다."}, {"url": "https://www.president.go.kr/president/greeting", "content": "대한민국 대통령 > 윤석열 대통령 > 취임사 대한민국 대통령 대한민국 대통령 대한민국 대통령 저는 자유, 인권, 공정, 연대의 가치를 기반으로 국민 여러분, 그러나 우리는 할 수 있습니다. 우리는 자유의 가치를 제대로, 그리고 정확하게 인식해야 합니다. 우리 사회 모든 구성원이 자유 시민이 되어야 하는 것입니다. 자유 시민이 되기 위해서는 일정한 수준의 경제적 기초, 그리고 공정한 교육과 문화의 도약과 빠른 성장은 오로지 과학과 기술, 그리고 혁신에 의해서만 이뤄낼 수 있는 것입니다. 그리고 평화는 자유와 인권의 가치를 존중하는 국제사회와의 연대에 의해 보장이 됩니다. 일시적으로 전쟁을 회피하는 취약한 평화가 아니라 자유와 번영을 꽃피우는 지속 가능한 평화를 추구해야 합니다. 우리나라뿐 아니라 세계 시민 모두의 자유와 인권을 지키고 확대하는데 더욱 주도적인 역할을 해야 합니다. 저는 자유, 인권, 공정, 연대의 가치를 기반으로 국민이 진정한 주인인 나라, 대한민국 대통령"}]
Assistant: 대한민국의 현직 대통령은 제20대 윤석열입니다. 그는 2022년 5월 10일부터 임기를 시작하였으며, 임기는 2027년 5월 9일까지입니다. 

더 자세한 내용은 [여기](https://namu.wiki/w/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%20%EB%8C%80%ED%86%B5%EB%A0%B9)에서 확인할 수 있습니다.
Use

In [ ]:
snapshot = graph.get_state(config)
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='대한민국의 대통령은 누구야?', additional_kwargs={}, response_metadata={}, id='409c71f7-920a-4a62-b139-595c8244e329'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_leB5386OKL3RtbUivihsnJEc', 'function': {'arguments': '{"query":"2023 대한민국 대통령"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 87, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-60439dd2-06f8-4bd3-bf04-3e2d8a5fe164-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2023 대한민국 대통령'}, 'id': 'call_leB5386OKL3RtbUivihsnJ

In [ ]:
config2 = {"configurable": {"thread_id": "2"}}
graph.invoke({"messages": [{"role": "user", "content": "내가 한 첫 질문이 뭐였어?"}]}, config2)

{'messages': [HumanMessage(content='내가 한 첫 질문이 뭐였어?', additional_kwargs={}, response_metadata={}, id='881a056d-d162-4a27-ac9c-c0268e3276ea'),
  AIMessage(content='죄송하지만, 이전 대화 내용을 기억할 수는 없습니다. 처음 질문이 무엇인지 알려주시면 도와드리겠습니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 89, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-57c4cb86-3990-4d1e-94e2-2b9823f4149b-0', usage_metadata={'input_tokens': 89, 'output_tokens': 30, 'total_tokens': 119, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

### 기억할 메세지 개수 제한하기

In [ ]:
def filter_messages(messages: list):
    return messages[-2:]

# chatbot 함수 수정
def chatbot(state: State):
    messages = filter_messages(state["messages"])  # 마지막 2개 유지
    result = llm_with_tools.invoke(messages)  # 필터된 메시지만 사용
    return {"messages": messages + [result]}  # 결과 추가

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
state = {"messages": []}

config = {"configurable": {"thread_id": "13"}}

# 첫 번째 입력
input_message = HumanMessage(content="hi! I'm bob and I like soccer")
state["messages"].append(input_message)  # 상태 업데이트
for event in graph.stream(state, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# 두 번째 입력
input_message = HumanMessage(content="what's my name?")
state["messages"].append(input_message)  # 상태 업데이트
for event in graph.stream(state, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

# 세 번째 입력
input_message = HumanMessage(content="What's my favorite?")
state["messages"].append(input_message)  # 상태 업데이트
for event in graph.stream(state, config, stream_mode="values"):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

hi! I'm bob and I like soccer
================================== Ai Message ==================================

Hi Bob! It's great to meet a fellow soccer fan. Do you have a favorite team or player?
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is Bob.
================================ Human Message =================================

What's my favorite?
================================== Ai Message ==================================

I don't have information about your preferences or favorites. If you let me know what you're referring to, I can help you find more information or suggest something!
